# SUPG Stabilization
In this notebook we introduce an important type of stabilization method for space–time finite element schemes.  
We consider the time-dependent convection–diffusion problem

$$
\nu\,(-\Delta u) \;+\; w \cdot \nabla u \;+\; \partial_t u \;=\; f(x,t)
\qquad \text{in } \Omega \times [0,T],
$$

where the spatial domain is the cube

$$
\Omega = [0,1] \times [0,1] \times [0,1].
$$

On the spatial boundary we impose a time-dependent Dirichlet condition

$$
u(x,t) = u_D(t),
\qquad x \in \partial\Omega,\; t \in [0,T].
$$

and we prescribe initial conditions 
$$
u(x,0) = u_0(x) 
\qquad x \in \Omega, t=0 
$$  
On the spatial boundary we impose a time-dependent Dirichlet condition

$$
u(x,t) = u_D(t),
\qquad x \in \partial\Omega,\; t \in [0,T].
$$


This formulation combines diffusion, convection, and the time derivative into a unified space–time PDE.  
It will serve as the base problem for demonstrating how stabilization becomes essential when convection or the time derivative dominates the behavior of the solution.


In [1]:
from import_hack import *
from methodsnm.mesh_4d import *
from methodsnm.visualize import *
from methodsnm.solver import solve_on_freedofs
from methodsnm.forms import *
from methodsnm.formint import *
import numpy as np
from netgen.csg import unit_cube
from ngsolve import Mesh,VOL,specialcf
from methodsnm.fes import *
from numpy import exp

source module for methodsNM imported.


# FEM
First, we create our 4D mesh.  
We use a semi-structured 4D mesh consisting of P1 simplices (pentatopes).


In [2]:
def list_diff(a, b):
    """Entfernt alle Elemente aus Liste a, die in Liste b enthalten sind."""
    return [x for x in a if x not in b]

ngmesh = Mesh(unit_cube.GenerateMesh(maxh=0.5))
T =4
mesh = UnstructuredHypertriangleMesh(T,ngmesh)
fes = P1_Hypertriangle_Space(mesh)


## Solving the Convection–Diffusion Problem in 4D with Standard CG

We now solve the time-dependent convection–diffusion problem

$$
\partial_t u - \varepsilon \,\Delta_x u + w \cdot \nabla_x u = f(x,t)
\quad \text{in } \Omega \times [0,T],
$$

with spatial domain

$$
\Omega = [0,1] \times [0,1] \times [0,1],
$$

for several values $\varepsilon \to 0$ to enter the convection-dominated regime.

### Exact Solution

We choose a space–time exact solution of the form

$$
u(x,t) = \phi(x)\,e^{-t},
$$

where

$$
\phi(x) 
= \frac{1 - \exp\!\left(\dfrac{x_0 + x_1 + x_2 - 3}{\varepsilon}\right)}
       {1 - \exp\!\left(-\dfrac{1}{\varepsilon}\right)}.
$$

This creates a sharp boundary layer near the corner $(1,1,1)$ for small $\varepsilon$.

### Forcing Term

With this choice of $u(x,t)$, we obtain

$$
\partial_t u(x,t) = -\phi(x)\,e^{-t},
$$

and we construct the right-hand side such that only the time derivative remains, i.e.

$$
f(x,t) = \partial_t u(x,t) -\varepsilon \Delta_x u + \cdot \nabla_x u
       = -\phi(x)\,e^{-t}
       = -\frac{1 - \exp\!\left(\dfrac{x_0 + x_1 + x_2 - 3}{\varepsilon}\right)}
               {1 - \exp\!\left(-\dfrac{1}{\varepsilon}\right)}
         \, e^{-t}.
$$

In code, using the space–time coordinate $x = (x_0,x_1,x_2,t)$, this corresponds to

```python
f = lambda x: -(1 - np.exp((x[0] + x[1] + x[2] - 3)/epsi)) \
              / (1 - np.exp(-1/epsi)) * np.exp(-x[3])


In [3]:
epsilon =[0.1, 0.01, 0.001]
for epsi in epsilon:
    wind = 1
    eps = GlobalFunction(lambda x: epsi, mesh = mesh)
    w = ConstantVectorFunction(np.array([1,1,1,1]), mesh = mesh)

    blf = BilinearForm(fes)
    blf += LaplaceIntegral_without_time(eps)
    blf += ConvectionIntegral(w)
    blf.assemble()

    f = lambda x: -(1 - exp((x[0]+x[1]+x[2] - 3)/epsi)) / (1 - exp(-1/epsi))* exp(-x[3])
    f = GlobalFunction(f, mesh = mesh)
    lf = LinearForm(fes)
    lf += SourceIntegral(f)
    lf.assemble()

    top = mesh.top_bndry_vertices
    initial = mesh.initial_bndry_vertices
    bndry = list_diff(mesh.bndry_vertices, top)
    freedofs = list_diff(mesh.vertices,bndry)

    uex = lambda x: (1 - exp((x[0]+x[1]+x[2] - 3)/epsi)) / (1 - exp(-1/epsi)) * exp(-x[3])
    uh = FEFunction(fes)
    uh._set(uex, bndry)

    res = lf.vector - blf.matrix.dot(uh.vector)
    uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
    from methodsnm.forms import compute_difference_L2
    u_exact = GlobalFunction(uex, mesh = mesh)
    l2diff = compute_difference_L2(uh, u_exact, mesh, intorder = 5)
    print("For epsilon :", epsi,"L2 difference  :" ,l2diff)

For epsilon : 0.1 L2 difference  : 0.025995964232032355
For epsilon : 0.01 L2 difference  : 0.03185591825883413
For epsilon : 0.001 L2 difference  : 0.20838205298991297


### SUPG Stabilization
So we see that for $\epsilon$ very small we get problems with our L2 error. One way to stabilize this problem is using an SUPG term. 

We use the following space–time SUPG stabilization terms. For the bilinear form we add

$$
a_{\text{SUPG}}(u,v)
:= \sum_{Q \in \mathcal{Q}_h} \gamma_{\text{SUPG}}
\int_{Q} \bigl( \partial_t u - \varepsilon \Delta u + w \cdot \nabla u \bigr)
       \bigl( w \cdot \nabla v + \partial_t v \bigr)\,dQ,
$$

and for the right-hand side

$$
f_{\text{SUPG}}(v)
:= \sum_{Q \in \mathcal{Q}_h} \gamma_{\text{SUPG}}
\int_{Q} f \,\bigl( w \cdot \nabla v + \partial_t v \bigr)\,dQ.
$$

The stabilization parameter is defined by

$$
\gamma_T
= \left(
  \begin{pmatrix} 1 \\ w \end{pmatrix}^\top
  G\,
  \begin{pmatrix} 1 \\ w \end{pmatrix}
  + C_{\mathrm{inv}}^2\,\varepsilon\,h^{-2}
\right)^{-1/2},
$$

where $G$ denotes the element-wise metric tensor in space–time, $h$ is a characteristic mesh size, and $C_{\mathrm{inv}}$ is the inverse inequality constant.

The concrete implementation can be found in the SUPG routine in the `formint` module (see the corresponding `SUPG` integrator). A detailed analysis of this stabilization term will be given in the forthcoming bachelor thesis; here we restrict ourselves to its numerical behavior in the presented test cases.


In [4]:
blf += SUPGIntegral(w)
blf.assemble()

lf += SUPGSourceIntegral(f, w)
lf.assemble()

uh = FEFunction(fes)
uh._set(uex, bndry)

res = lf.vector - blf.matrix.dot(uh.vector)
uh.vector += solve_on_freedofs(blf.matrix,res,freedofs)
from methodsnm.forms import compute_difference_L2
u_exact = GlobalFunction(uex, mesh = mesh)
l2diff = compute_difference_L2(uh, u_exact, mesh, intorder = 5)
print("For epsilon :", epsi,"L2 difference  :" ,l2diff)

For epsilon : 0.001 L2 difference  : 0.03156551003930699


As you can now easily see the SUPG stabilization helped us to get a better result